# Práctica 3 - Multiclasificadores y selección de variables



## Minería de Datos 2017/2018 - Jacinto Arias y José A. Gámez

Dividiremos esta práctica en dos partes:

**Multiclasificadores:** estudiaremos la API de scikit para los modelos de tipo ensemble y veremos como entrenar, seleccionar y utilizar estos modelos.

**Selección de variables:** veremos los distintos algoritmos de selección de variables disponibles en scikit y como aplicarlos.

In [1]:
# Always load all scipy stack packages
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(color_codes=True)

In [2]:
# This code configures matplotlib for proper rendering
%matplotlib inline
mpl.rcParams["figure.figsize"] = "8, 4"
import warnings
warnings.simplefilter("ignore")

In [3]:
import base64

---

In [4]:
seed=6342
np.random.seed(seed)

## Carga de datos

Esta vez vamos a utilizar los datos obtenidos directamente del benchmark de scikit learn [Docs](http://scikit-learn.org/stable/datasets/index.html#optical-recognition-of-handwritten-digits-data-set). Concretamente utilizaremos la versión completa del dataset Wisconsin.

In [5]:
from sklearn.datasets import load_breast_cancer
wisconsin = load_breast_cancer()

Estos datasets vienen preparados directamente para la clasificación y separan en un diccionario los distintos elementos que nos puedan interesar. Si quisieramos recuperar los datos en un data.frame tendríamos que combinar los elementos correctamente:

In [6]:
wisconsin.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [7]:
df = pd.DataFrame(
    data = np.column_stack((wisconsin.data, wisconsin.target)), 
    columns = np.append(wisconsin.feature_names, "label")
)

In [8]:
df.sample(5)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,label
495,14.87,20.21,96.12,680.9,0.09587,0.08345,0.06824,0.04951,0.1487,0.05748,...,28.48,103.90,783.6,0.1216,0.13880,0.17000,0.10170,0.2369,0.06599,1.0
67,11.31,19.04,71.80,394.1,0.08139,0.04701,0.03709,0.02230,0.1516,0.05667,...,23.84,78.00,466.7,0.1290,0.09148,0.14440,0.06961,0.2400,0.06641,1.0
173,11.08,14.71,70.21,372.7,0.10060,0.05743,0.02363,0.02583,0.1566,0.06669,...,16.82,72.01,396.5,0.1216,0.08240,0.03938,0.04306,0.1902,0.07313,1.0
500,15.04,16.74,98.73,689.4,0.09883,0.13640,0.07721,0.06142,0.1668,0.06869,...,20.43,109.70,856.9,0.1135,0.21760,0.18560,0.10180,0.2177,0.08549,1.0
406,16.14,14.86,104.30,800.0,0.09495,0.08501,0.05500,0.04528,0.1735,0.05875,...,19.58,115.90,947.9,0.1206,0.17220,0.23100,0.11290,0.2778,0.07012,1.0


Como podéis observar, la variable `label` no está codificada como categories, sino como una variable contínua. Esta es la forma preferida de representación en la mayoría de librerías de Machine Learning, ya que optimiza el indexado y otras operaciones. No obstante, hemos de tener cuidado para asegurarnos que todos los algoritmos y técnicas que utilicemos entienden que se trata de un problema de clasificación y no de regresion, ya que en este caso no hay forma de distinguirlo.

Puede que en algunos casos queramos disponer de la variable clase como una categoría, especialmente para realizar análisis exploratorio y generar gráficas, para ello necesitamos utilizar las herramientas de pandas para representar datos categóricos.

**Nota:** Si buscáis sobre este problema lo más común es encontrar soluciones al problema contrario, como codificar mis variables categóricas como numéricas o en general hacer *scrapping* a partir de strings.

In [9]:
target_categorical = pd.Series(
     pd.Categorical([wisconsin.target_names[x] for x in wisconsin.target], 
     categories=wisconsin.target_names)
)

dfLabel = pd.DataFrame(target_categorical, columns=["label"])
dfLabel.sample(3)

,label
287,benign
55,benign
455,benign


Independientemente, realizaremos el resto de la práctica utilizando las versiones numéricas de los datos para demostrar su versatilidad.

In [10]:
attributes = wisconsin.data
label = wisconsin.target

---

## Ensembles en scikit-learn

La librería implementa una gran colección de las técnicas más populares de modelos ensemble. Os recomiendo la lectura del apartado correspondiente de la documentación ya que esclarece bastante bien los diversos detalles de implementación de cada algoritmo. Podéis hacerlo en el siguiente [Enlace](http://scikit-learn.org/stable/modules/ensemble.html#).

### Decision tree como referencia

Para poder estudiar la efectividad de los algoritmos anteriores evaluaremos un árbol de decisión sin poda.

In [11]:
# Cargamos el arbol de decision
from sklearn import tree
from sklearn.model_selection import cross_val_score

dt = tree.DecisionTreeClassifier()
scores_dt = cross_val_score(estimator=dt, X=attributes, y=label, scoring="accuracy", cv=3)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_dt.mean(), scores_dt.std() * 2))

Accuracy: 0.91 (+/- 0.03)


## Bagging

La estrategia básica tras el algoritmo de **bagging** es la agregación de distintos clasificadores que han sido aprendidos a partir de muestras obtenidas tras un proceso de bootstraping (muestreo con remplazo). Aquí estudiaremos cómo aprender un modelo de este tipo utilizando scikit.

```
sklearn.ensemble.BaggingClassifier(base_estimator=None, (El clasificador de base que usará el ensemble) 
                                   n_estimators=10, (Numero de modelos que aprenderemos) 
                                   max_samples=1.0, (proporción de la muestra a utilizar)
                                   max_features=1.0, (proporcion de características a  utilizar)
                                   bootstrap=True, (si se utilizará muestreo por remplazo o no)
                                   n_jobs=1, (para utilizar paralelismo, debe soportarlo nuestro entorno)
                                   random_state=None (la semilla)
                                  )
```

A continuación aprenderemos un ensemble de árboles de decisión mediante bagging, para ello, hemos de suministrarle el modelo a utilizar, en nuestro caso: `tree.DecisionTreeClassifier()`

In [12]:
from sklearn.ensemble import BaggingClassifier

bagg = BaggingClassifier(tree.DecisionTreeClassifier(), 
                          n_estimators = 30, 
                          random_state=seed)

In [13]:
scores_bagg = cross_val_score(bagg, attributes, label, cv=3, scoring="accuracy")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_bagg.mean(), scores_bagg.std() * 2))

Accuracy: 0.93 (+/- 0.04)


## Random Forest
Una alternativa muy popular al bagging es el clasificador **random forest**. En este caso también se utiliza muestreo con remplazo, pero adicionalmente integra otras técnicas de aleatorización en el aprendizaje de los árboles de decisión para ampliar la generalización del ensemble. Concretamente utiliza una muestra aleatoria de los atributos a la hora de seleccionar cada punto óptimo de corte.

Un clasificador random forest siempre utiliza árboles de decisión como submodelos y por ello no requiere un clasificador base para su definición. Por esa misma razón los hyperparámetros de este clasificador incluyen tanto los correspondientes al ensemble como al árbol de decisión.

```
sklearn.ensemble.RandomForestClassifier(n_estimators=10, (numero de modelos que aprenderemos)
                                        criterion='gini', (parametro del arbol de decision)
                                        max_depth=None, (parametro del arbol de decision)
                                        min_samples_split=2, (parametro del arbol de decision)
                                        min_samples_leaf=1, (parametro del arbol de decision)
                                        min_impurity_split=1e-07, (parametro del arbol de decision)
                                        max_features='auto', (número de atributos que se usan en cada caso:
                                                              puede ser un entero para número exacto, un float
                                                              para proporcion o 'auto', 'sqrt', 'log2' o 'None')
                                        bootstrap=True, (si se utilizará muestreo por remplazo)
                                        n_jobs=1, (para utilizar paralelismo, debe soportarlo nuestro entorno)
                                        random_state=None, (semilla)
                                        )
```

In [14]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=30, random_state=seed)

In [15]:
scores_rf = cross_val_score(rf, attributes, label, cv=3, scoring="accuracy")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rf.mean(), scores_rf.std() * 2))

Accuracy: 0.95 (+/- 0.03)


## Boosting
La estrategia de boosting es completamente diferente a la anterior, ya que en lugar de reducir la varianza al aprender múltiples clasificadores independientes, realizaremos un proceso iterativo en el que intensificaremos el aprendizaje sobre aquellas instancias más complicadas de aprender para nuestro modelo. 
El algoritmo más conocido se llama `AdaBoost` y está disponible en scikit:

```
sklearn.ensemble.AdaBoostClassifier(base_estimator=None, (El clasificador de base que usará el ensemble) 
                                    n_estimators=50, (numero de modelos que aprenderemos)
                                    learning_rate=1.0, (la contribución del modelo anterior al siguiente)
                                    random_state=None (semilla)
                                    )
```

In [25]:
from sklearn.ensemble import AdaBoostClassifier
#boost = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(), n_estimators=30)
boost = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=3), n_estimators=30)

In [26]:
scores_boost = cross_val_score(boost, attributes, label, cv=3, scoring="accuracy")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_boost.mean(), scores_boost.std() * 2))

Accuracy: 0.97 (+/- 0.02)


## Gradient Boosting Trees
Es una generalización del algoritmo de boosting que optimiza funciones de coste en el dominio concreto de los árboles de decisión. Actualmente es una de las técnicas más exitosas para resolver problemas de clasificación complejos *off of the shelf*.

```
sklearn.ensemble.GradientBoostingClassifier(n_estimators=100, (numero de modelos que aprenderemos)
                                            learning_rate=0.1, (la contribución del modelo anterior)
                                            subsample=1.0, (aprender mediante submuestreo)
                                            max_features=None, (submuestrear atributos como en RF)
                                            min_samples_split=2, (parametro del arbol de decision)
                                            min_samples_leaf=1, (parametro del arbol de decision)
                                            max_depth=3, (parametro del arbol de decision)
                                            min_impurity_split=1e-07, (parametro del arbol de decision)
                                            random_state=None, (semilla)
                                            )
```

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
gboost = GradientBoostingClassifier(n_estimators=30)

In [19]:
scores_gboost = cross_val_score(gboost, attributes, label, cv=5, scoring="accuracy")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_gboost.mean(), scores_gboost.std() * 2))

Accuracy: 0.95 (+/- 0.03)


## Comparación
Cualquier técnica de ensemble debería ser capaz de mejorar el rendimiento del clasificador base, aunque el rendimiento entre ellas puede depender enormemente del **problema** y de los **parámetros** con los que se hayan configurado las técnicas.

In [20]:
# How to make a HTML table!
from IPython.display import display, HTML

def printTable(list):
    table = """<table>%s</table>"""
    row = """<tr>%s</tr>"""
    cell = """<td>%s</td>"""
    report =  table % ''.join([row % (cell % x[0] + cell % x[1]) for x in results])
    display(HTML(report))
    

In [27]:
results = (("Decision Tree", scores_dt.mean()), 
           ("Bagging", scores_bagg.mean()), 
           ("Random Forest", scores_rf.mean()),
           ("Boosting", scores_boost.mean()),
           ("Gradient Boosting", scores_gboost.mean()))

printTable(results)

Decision Tree,0.906850459482
Bagging,0.934939199851
Random Forest,0.954311705189
Boosting,0.970138308735
Gradient Boosting,0.95444401693


### Estrategias de reducción de varianza vs estrategias de reducción de sesgo (variance and bias)

Aunque la estrategia común detrás de un ensemble sea la de generar un consenso entre varios modelos (mediante agregación o voto por ejemplo), la forma en la que se genera el conjunto de modelos está dirigida por motivaciones muy diferentes para algoritmos como bagging o boosting. Cuando utilizamos una u otra técnica es fundamental conocer el fundamento interno de la técnica de ensembling para seleccionar y aprender correctamente los modelos del ensemble.

En el caso de **bagging** o **random forest** queremos utilizar una función de aprendizaje (C4.5, CART...) y obtener modelos "diversos" a partir de los mismos datos para reducir el error obtenido mediante **varianza**. Formalmente hablamos de clasificadores que individualmente tengan poder de generalización pero que al mismo tiempo estén lo menos correlados entre ellos. Es por ello que se utilizan diversas técnicas de submuestreo y aleatorización de los modelos. 

En el caso de los algoritmos de **boosting** la estrategia es opuesta, dado un problema complejo queremos reducir el sesgo en nuestros modelos. Por ello en lugar de aleatorización guíamos la función de aprendizaje para que incida en aquellos ejemplos que sean más dificiles de generalizar. La técnica general es utilizar sobremuestreo o pesos para que sobreajustar el clasificador en aquellos casos conflictivos, de manera más formal o sofisticada se puede establecer un problema de optimización de funciones de coste.

Para enteder este problema hay que saber responder a las siguientes preguntas:

##### ¿Cuáles son los mejores clasificadores para ejecutar algoritmos de bagging?

In [22]:
base64.b64encode(b'Weak learners, o clasificadores con un poder de generalizacion y parametros no muy complejos ni sobreajustados. Esto otorga al algoritmo de boosting espacio para optimizar dichos parametros y teoricamente transformar al clasificador base en un strong learner. Por ejemplo, arboles de decision muy poco profundos (shallow trees)')

b'V2VhayBsZWFybmVycywgbyBjbGFzaWZpY2Fkb3JlcyBjb24gdW4gcG9kZXIgZGUgZ2VuZXJhbGl6YWNpb24geSBwYXJhbWV0cm9zIG5vIG11eSBjb21wbGVqb3Mgbmkgc29icmVhanVzdGFkb3MuIEVzdG8gb3RvcmdhIGFsIGFsZ29yaXRtbyBkZSBib29zdGluZyBlc3BhY2lvIHBhcmEgb3B0aW1pemFyIGRpY2hvcyBwYXJhbWV0cm9zIHkgdGVvcmljYW1lbnRlIHRyYW5zZm9ybWFyIGFsIGNsYXNpZmljYWRvciBiYXNlIGVuIHVuIHN0cm9uZyBsZWFybmVyLiBQb3IgZWplbXBsbywgYXJib2xlcyBkZSBkZWNpc2lvbiBtdXkgcG9jbyBwcm9mdW5kb3MgKHNoYWxsb3cgdHJlZXMp'

In [23]:
# Run me to know the answer (b64 encoded quizes FTW!)
base64.b64decode(b'U3Ryb25nIGxlYXJuZXJzLCBjbGFzaWZpY2Fkb3JlcyBjb24gdW4gZ3JhbiBwb2RlciBwcmVkaWN0aXZvIGVuIHN1cyBwYXJhbWV0cm9zIGRhbmRvIHBpZSBhIHV0aWxpemFyIG1vZGVsb3MgcXVlIHByZXNlbnRlbiBtdWNoYSB2YXJpYW56YSwgeWEgcXVlIGVzdGEgc2UgcmVkdWNlLCBjb21vIGVzIGVsIGNhc28gZGUgYXJib2xlcyBkZSBkZWNpc2lvbiBzaW4gcG9kYXIuIA==')

b'Strong learners, clasificadores con un gran poder predictivo en sus parametros dando pie a utilizar modelos que presenten mucha varianza, ya que esta se reduce, como es el caso de arboles de decision sin podar. '

##### ¿Cuáles son los mejores clasificadores para ejecutar algoritmos de boosting?

In [24]:
# Run me to know the answer (b64 encoded quizes FTW!)
base64.b64decode(b'V2VhayBsZWFybmVycywgbyBjbGFzaWZpY2Fkb3JlcyBjb24gdW4gcG9kZXIgZGUgZ2VuZXJhbGl6YWNpb24geSBwYXJhbWV0cm9zIG5vIG11eSBjb21wbGVqb3Mgbmkgc29icmVhanVzdGFkb3MuIEVzdG8gb3RvcmdhIGFsIGFsZ29yaXRtbyBkZSBib29zdGluZyBlc3BhY2lvIHBhcmEgb3B0aW1pemFyIGRpY2hvcyBwYXJhbWV0cm9zIHkgdGVvcmljYW1lbnRlIHRyYW5zZm9ybWFyIGFsIGNsYXNpZmljYWRvciBiYXNlIGVuIHVuIHN0cm9uZyBsZWFybmVyLiBQb3IgZWplbXBsbywgYXJib2xlcyBkZSBkZWNpc2lvbiBtdXkgcG9jbyBwcm9mdW5kb3MgKHNoYWxsb3cgdHJlZXMp')

b'Weak learners, o clasificadores con un poder de generalizacion y parametros no muy complejos ni sobreajustados. Esto otorga al algoritmo de boosting espacio para optimizar dichos parametros y teoricamente transformar al clasificador base en un strong learner. Por ejemplo, arboles de decision muy poco profundos (shallow trees)'

Si observamos la configuración de los experimentos anteriores podemos ver que en el caso de random forest, el árbol de decisión no está podado por defecto mientras que en gboost los árboles tienen una profundidad máxima de 3. Este el motivo por el que el algoritmo básico de boosting no esta funcionando correctamente. Probad a modificar los parámetros de los meta-algoritmos y del algoritmo base para confirmar el comportamiento anterior

#### Convergencia respecto al número de modelos

Una duda común en el uso de técnicas de *ensembling* surge a la hora de fijar el número de modelos a aprender. Es importante conocer como se comportan los clasificadores conforme el número de modelos aumenta. Ambos clasificadores convergen de una forma diferente.

En el caso de bagging, para cada problema y configuración, el clasificador se estabiliza conforme se agregan más modelos al ensemble hasta llegar a un punto de convergencia en el cual la varianza se ha reducido "todo lo posible" y no mejorarán los resultados. La moraleja es que un número elevado de modelos mejorará los resultados, pero que siempre hay un punto en el que hay "suficientes", pasar de este punto no empeorará teóricamente nuestro modelo pero sí aumentará enormente el tiempo de aprendizaje y clasificación.

En el caso de boosting el problema es más complejo, y las connotaciones teóricas tienen mayor implicación. Al tratarse de un problema de optimización el algoritmo converge de una forma diferente ya que sobreajusta a los datos. Es por ello que es necesario elegir mejor el número de modelos, no solo para limitar el número de recursos que necesitemos sino también para no sobreajustar demasiado los datos.

### Complejidad temporal (y espacial) en ensembles

Como hemos anticipado, los meta-clasificadores basados en ensembles son muy potentes. Esta mejora en la clasificación la ganamos a cambio de recursos computacionales, ya que multiplicamos directamente el número de recursos necesarios en comparación con un algoritmo de clasificación básica.

La complejidad viene acotada por el número de modelos, por lo que la cantidad de recursos de los que dispongamos tendrá que ser un dactor determinante a la hora de fijar este valor.

**Paralelismo:** Cada día es más común encontrar plataformas de cómputo paralelo, especialmente desde la explosión del Big Data. En este caso, los algoritmos de boosting pueden ser fácilmente adaptados para ser optimizados en paralelo, mientras que los algoritmos de boosting, al ser procesos iterativos, no se benefician de las mismas ventajas. 

In [28]:
# TRAINING TIME

print("\nDecision Tree")
model = tree.DecisionTreeClassifier()
%timeit -n 5 model.fit(attributes, label)

print("\nRandom Forest")
model = RandomForestClassifier(n_estimators=100, random_state=1234)
%timeit -n 5 model.fit(attributes, label)

print("\nGradient Boosting")
model = GradientBoostingClassifier(n_estimators=100)
%timeit -n 5 model.fit(attributes, label)


Decision Tree
10.5 ms ± 1.87 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)

Random Forest
272 ms ± 23.1 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)

Gradient Boosting
246 ms ± 21.1 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


---------------------------------------------------------------------------------

# Selección de variables en scikit-learn

Tal y como hemos visto en clase, existen distintas estrategias a la hora de reducir la dimensionalidad de un problema. Aunque scikit incorpora una serie de algoritmos y métricas de selección de variables [Docs](http://scikit-learn.org/stable/modules/feature_selection.html), su catálogo es limitado con respecto al estado del arte actual en la literatura.

### Métodos filter univariados

### Eliminación de variables con baja varianza

Resulta obtvio que si una variable presenta valores de varianza muy reducidos (por ejemplo una constante) no será un buen atributo predictor. Aunque este método pertenece más a la familia de métodos de preprocesamiento y eliminación de ruido sklearn lo clasifica como un método de preprocesado y así lo incluye en su API.

```
VarianceThreshold(
    threshold = 0.0 # Valor mínimo de la varianza para considerarse ruidoso
```

Como veréis a continuación, los métodos de selección de variables presentan la misma API que cualquier algoritmo de transformación:

In [29]:
from sklearn.feature_selection import VarianceThreshold

# We add a noisy constant column
noise = np.ones(attributes.shape[0])
noisy_atts = np.column_stack((attributes, noise))
print( noisy_atts.shape )

# We add a noisy constant column
sel = VarianceThreshold()
sel.fit(noisy_atts)
print( sel.transform(noisy_atts).shape )

(569, 31)
(569, 30)


### Selección por rankings

Scikit implementa el algoritmo filter más sencillo de todos, la selección por rankings. Para ello ordena todas los atributos dada una métrica y selecciona los `k`mejores para el clasificador. Distintas versiones del algoritmo nos permiten especificar `k`como una constante o una proporción.

Para ejecutar este algoritmo necesitaremos no solo la estrategia de ranking sino la implementación de la métrica correspondiente. Scikit provee algunas de las más comunes como la información mútua.

```
sklearn.feature_selection.mutual_info_classif   # Información mútua
```

In [30]:
from sklearn.feature_selection import mutual_info_classif

# We can compute the scores and ranking directly:

scores = list(mutual_info_classif(attributes, label, random_state=seed))
names = list(wisconsin.feature_names)
ranks = sorted( list(zip(scores, names)), reverse=True )
ranks

[(0.47518525067526252, 'worst perimeter'),
 (0.4630635297199055, 'worst area'),
 (0.45515685867696409, 'worst radius'),
 (0.44046272810851894, 'mean concave points'),
 (0.43785096902449139, 'worst concave points'),
 (0.40429591576647761, 'mean perimeter'),
 (0.37435430813985771, 'mean concavity'),
 (0.36957310392853637, 'mean radius'),
 (0.35859977750084582, 'mean area'),
 (0.33913602338779492, 'area error'),
 (0.31686394124406769, 'worst concavity'),
 (0.27361385712564101, 'perimeter error'),
 (0.24870957033219399, 'radius error'),
 (0.22752331946337234, 'worst compactness'),
 (0.21223646713681044, 'mean compactness'),
 (0.12794646349638605, 'concave points error'),
 (0.11960002324391539, 'concavity error'),
 (0.11899047482845848, 'worst texture'),
 (0.096465466606317385, 'worst smoothness'),
 (0.095982484270855428, 'mean texture'),
 (0.092908406557946632, 'worst symmetry'),
 (0.076624709670718216, 'compactness error'),
 (0.075970717512003905, 'mean smoothness'),
 (0.07199972257453013

Podemos utilizar esta métrica en nuestro análisis exploratorio si lo deseamos, no obstante, como algoritmo de selección de variables es fundamental que integremos la selección dentro de nuestro proceso de selección, por lo que debemos utilizar un método de transformación:

```
sklearn.feature_selection.SelectKBest(
    score_func  # Métrica a optimizar
    k=10 # Número de atributos a seleccionar
)
```

In [31]:
from sklearn.feature_selection import SelectKBest
# We configure and train the algorithm
fss = SelectKBest(mutual_info_classif, k=5).fit(attributes, label)

# We can obtain the mask of the selected attributes
print(wisconsin.feature_names[fss.get_support()])

# We can transform the dataset to project only the selected attributes
print(fss.transform(attributes).shape)

['mean concave points' 'worst radius' 'worst perimeter' 'worst area'
 'worst concave points']
(569, 5)


### Métodos wrapper

### Métricas de importancia de las variables

Algunos clasificadores pueden asignar una métrica de importancia a sus variables a partir de los parámetros obtenidos durante el algoritmo de aprendizaje. Por ejemplo, en una regresión podemos fijarnos en los pesos de los coeficientes de las variables. Una de las métricas más utilizadas es la métrica de importancia obtenida en un ensemble de árboles, donde la importancia se calcula a partir de la influencia media que tiene cada variable en el proceso de construcción de los árboles (en función de cuántas veces es seleccionada y a cuantas instancias implica).

Esta métrica puede utilizarse para establecer un ranking entre las variables para seleccionar un subconjunto óptimo. Este proceso puede realizarse de manera independiente al aprendizaje el clasificador, por lo que podemos combinar modelos.



In [32]:
model = RandomForestClassifier(n_estimators=100, random_state=seed)
classifier = model.fit(attributes, label)

# We can obtain the importances for any model capable of computing them
scores = classifier.feature_importances_
names = list(wisconsin.feature_names)
ranks = sorted( list(zip(scores, names)), reverse=True )
ranks

[(0.13280182304084834, 'worst area'),
 (0.12522441553801289, 'worst concave points'),
 (0.11957281896310926, 'mean concave points'),
 (0.11305112291391831, 'worst radius'),
 (0.10725106470402063, 'worst perimeter'),
 (0.065995111613632348, 'mean concavity'),
 (0.055120669821861999, 'mean perimeter'),
 (0.046758683491729985, 'worst concavity'),
 (0.033865123475926698, 'area error'),
 (0.033228454027190352, 'mean area'),
 (0.02712100085555047, 'mean radius'),
 (0.023688538525107553, 'worst texture'),
 (0.015157495706731283, 'mean texture'),
 (0.012474648157482209, 'worst smoothness'),
 (0.011416966382167066, 'mean compactness'),
 (0.010083105240270118, 'radius error'),
 (0.0082233080270945544, 'worst symmetry'),
 (0.0072876605422124438, 'worst compactness'),
 (0.0061961130203895976, 'concavity error'),
 (0.0054616896259530598, 'worst fractal dimension'),
 (0.0054390259088372191, 'mean smoothness'),
 (0.0049092939420526775, 'concave points error'),
 (0.0048022705043699009, 'perimeter erro

Scikit nos permite introducir esta información durante la fase de preprocesado mediante la funcion `SelectFromModel`. En este caso es importante que suministremos al algoritmo con un modelo aprendido:

In [33]:
from sklearn.feature_selection import SelectFromModel
model = RandomForestClassifier(n_estimators=100, random_state=seed)
fss = SelectFromModel(estimator = model, threshold = "mean").fit(attributes, label)
fss.transform(attributes).shape

(569, 9)

### Búsqueda recursiva

La alternativa más popular, aunque más costosa en recursos computacionales son los métodos wrapper de búsqueda. En este caso utilizaremos el rendimiento del propio clasificador en un proceso de validación cruzada a modo de score para guiar una búsqueda **greedy** o exhaustiva por el espacio de búsqueda.

Generalmente se utilizan dos alternativas:

* **Busqueda Backward**: Se comienza por un clasificador que utiliza todos los atributos y se van eliminando las peores variables hasta que el resultado no mejora.

* **Búsqueda Forward**: Se comienza con una sola variable y se van añadiendo las más prometedoras hasta que el resultado no mejora.

En ocasiones se guía este proceso de búsqueda utilizando un orden preestablecido entre las variables, por ejemplo, mediante la aplicación de un algoritmo de ranking.

Lamentablemente, scikit learn no implementa directamente estos algoritmos en su API de selección de variables. No obstante, su definición es muy sencilla ya que al tratarse de métodos púramente wrappers no tienen cabida dentro de un pipeline y podrían ejecutarse desde fuera del bucle de validación cruzada.

### Métodos filter multivariados
Desafortunadamente scikit learn no implementa ningún método de filtering multivariado, al contrario que otras librerías como **weka**. No obstante, no es dificil implementar estos métodos haciendo uso de los estadísticos que sí podemos computar. Como la aproximación de la información mútua (Battiti) o la correlación entre las variables. 

# Objetivos de la práctica

En esta práctica tendremos dos objetivos principales:

### A) Uso y ajuste de métodos basados en ensembles

* **A.1) Implementación básica de un ensemble de manera manual:**
    ```
        Pseudocodigo básico:
        Generar una lista de datos muestreados (con/sin remplazo, con/sin muestreo de atributos)
        Aprender un clasificador para cada muestra
        Obtener predicciones para cada modelo
        Obtener la moda de dichas predicciones
    ```
Implementar vuestro propio ensemble, con la API más similar posible a la de scikit learn (aquí seremos flexibles). Se valorará probar distintas estrategias de randomización y explicar el rendimiento de cada una.

--> funcion pandas sample: usarla en un bucle. Aprender un modelo en cada iteracion y guardarlos en un vector de modelos (Bagging simple) --> mejorarlo para obtener puntos extra (usar cv).
    
*  **A.2) Uso de Bagging, Boosting, Random Forest y Gradient Boosting**: Aprender y ajustar los parámetros de los distintos modelos para maximizar la clasificación. Comparar estos resultados con el ensemble implementado manualmente.
  
    

### B) Selección de variables

Al igual que en el caso anterior, se busca que el alumno intente **mejorar** los resultados obenidos utilizando clasificadores base o ensembles al aplicar técnicas de selección de variables.

* **B.1 Utilizar un método filter basado en rankings y la importancia de las variables para evaluar distintos subconjuntos** --> funciones ranking y importance usando un grid serach
* **B.2 Implementar al menos un algoritmo de búsqueda recursiva wrapper** --> bucle for, fuera de la cv: una iteracion de cv, hacer wrapper, otra iteracion, otro wrapper...
* **B.3 (Opcional, recomendable) Implementar un algoritmo greedy basado en métricas filter (información mútua). si no se puede integrar el algoritmo en el proceso de validación (cross validation) se puede probar mediante evaluación holdout** --> por ejemplo un batiti

Se deberán implementar los experimentos para poderlos ejecutar para distintos conjuntos de datos. Se reportarán como mínimo los resultados para wisconsin y pima.